## Collecting protocol fees across Balancer core pools on all networks
Spreadsheet as reference: https://docs.google.com/spreadsheets/d/1xwUPpbYq7woVOU9vQ8EB8MY75I-1mauTLyDVwvKUDKo/edit#gid=0
Collab: https://colab.research.google.com/drive/1vKCvcV5mkL1zwW3565kLSGkBEbt8NsoB?usp=sharing


In [5]:
# Query:
POOLS_SNAPSHOTS_QUERY = """
{{
  poolSnapshots(
    first: {first}
    skip: {skip}
    orderBy: timestamp
    orderDirection: desc
    block: {{ number: {block} }}
    where: {{ protocolFee_not: null }}
  ) {{
    pool {{
      address
      id
      symbol
    }}
    timestamp
    protocolFee
    swapFees
    swapVolume
    liquidity
  }}
}}
"""

In [7]:
import os
from typing import Dict
from typing import List
from typing import Optional

from dotenv import load_dotenv
from gql import Client
from gql import gql
from gql.transport.requests import RequestsHTTPTransport
from web3 import Web3

load_dotenv()
web3 = Web3(Web3.HTTPProvider(os.environ["ETHNODEURL"]))
BALANCER_GRAPH_URL = "https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-arbitrum-v2"

# Arb block time is .3 seconds
ARB_CHAIN_BLOCK_TIME = 0.3

block_now = web3.eth.block_number
# Given Arb block time, we want to look back 2 weeks:
block_2_weeks_ago = block_now - (2 * 7 * 24 * 60 * 60 / ARB_CHAIN_BLOCK_TIME)
# Fetch all the data from the balancer subgraph
def make_gql_client(url: str) -> Optional[Client]:
    transport = RequestsHTTPTransport(url=url, retries=3)
    return Client(
        transport=transport, fetch_schema_from_transport=True, execute_timeout=60
    )


def get_balancer_pool_snapshots(block: int) -> Optional[List[Dict]]:
    client = make_gql_client(BALANCER_GRAPH_URL)
    all_pools = []
    limit = 500
    offset = 0
    while True:
        result = client.execute(
            gql(POOLS_SNAPSHOTS_QUERY.format(first=limit, skip=offset, block=block)))
        all_pools.extend(result['poolSnapshots'])
        offset += limit
        if len(result['poolSnapshots']) < limit - 1:
            break
    return all_pools


pool_snapshots_now = get_balancer_pool_snapshots(web3.eth.block_number)
pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(block_2_weeks_ago))